In [1]:
import os, sys, time, copy
import numpy as np
import matplotlib.pyplot as plt
import pickle
import random

sys.path.append('../')
sys.path.append('../Lib')
sys.path.append('../Protocols')
sys.path.append('../Models')
import protocol_lib

from br1977 import BR1977
from ord2011JK_v1 import ORD2011
import model_response
import simulator_myokit
import myokit

In [2]:
def get_high_fitness(ga_result):
    best_individual = ga_result.generations[0][0]

    for i, gen in enumerate(ga_result.generations):
        best_in_gen = ga_result.get_high_fitness_individual(i)
        if best_in_gen.fitness > best_individual.fitness:            
            best_individual = best_in_gen

    return best_individual

In [3]:
steps_in_protocol = 4
step_types=['step', 'ramp']
step_duration_bounds=(5, 1000)
step_voltage_bounds=(-120, 60)
window=2
step_size=1
LIST_OF_CURRENTS = [ 'I_Na', 'I_NaL', 'I_to', 'I_CaL', 'I_Kr', 'I_Ks', 'I_K1' ]# ['I_Na', 'I_Kr', 'I_Ks', 'I_To', 'I_F', 'I_CaL', 'I_K1'] 

In [4]:
trial_conditions = "trial_steps_ramps_ORD2011_256_64_4_-120_60"
current = 'I_Na'
ga_result = pickle.load(open(f'ga_results/{trial_conditions}/ga_results_{current}_artefact_False', 'rb'))
best_individual = get_high_fitness(ga_result)
best_protocol = best_individual.protocol
best_fitness = best_individual.fitness
print(best_protocol)
print(best_fitness)

|RAMP: Voltage Start: -24.884265647446643, Voltage End: -107.35898059956352, Duration: 244.98109499829968| | |STEP: Voltage: -38.266220913694674, Duration: 663.2670320031141| | |STEP: Voltage: -119.88353380492657, Duration: 45.58985412063697| | |RAMP: Voltage Start: -30.35604144977756, Voltage End: -73.95635295816794, Duration: 407.8358525181555|
0.9994632967886729


In [5]:
m_myokit, p, s = myokit.load("../mmt-model-files/ohara-cipa-v1-2017_VC.mmt")
sim_myokit = simulator_myokit.Simulator(m_myokit, best_protocol, max_step=1.0, abs_tol=1e-8, rel_tol=1e-8, vhold=-87) 
tr = model_response.get_model_response_with_myokit( sim_myokit, best_protocol, prestep=2000 )
tr

In [6]:
scale = 1
max_contributions = tr.current_response_info.get_max_current_contributions(
                time=tr.t,
                window=window/scale,
                step_size=step_size/scale)
print(max_contributions)

  Current  Contribution  Time Start  Time End
0    I_Na      0.998815       954.0     956.0
1   I_NaL      0.552977        23.0      25.0
2    I_to      0.066091       979.0     981.0
3   I_CaL      0.074783        12.0      14.0
4    I_Kr      0.811899       905.0     907.0
5    I_Ks      0.001645       707.0     709.0
6    I_K1      0.999382       242.0     244.0


In [7]:
fasfa

NameError: name 'fasfa' is not defined

In [ ]:
model = None
if 'ORD2011' in trial_conditions:
    model = ORD2011(best_protocol)
elif 'BR1977' in trial_conditions:
    model = BR1977(best_protocol)  
    
tr = model_response.get_model_response_JK(model, best_protocol, prestep=5000)
tr

In [ ]:
scale = 1
max_contributions = tr.current_response_info.get_max_current_contributions(
                time=tr.t,
                window=window/scale,
                step_size=step_size/scale)
print(max_contributions)


In [ ]:
current_contributions = tr.current_response_info.get_current_contributions( time= model.times, window=window, step_size=step_size )

In [ ]:
current_contributions.iloc[242]

In [ ]:
tr.plot_currents_contribution(current, window=window, step_size=step_size, is_shown=True, title=current,
        saved_to=f'./ga_results/{trial_conditions}/{current}.svg')

In [ ]:
'''
Plot
'''
plot_li = ['command_voltages', 'I_ion']+LIST_OF_CURRENTS
fig, axes = plt.subplots(len(plot_li), 1, figsize=(10, 20))    
fig.suptitle('ORD2011', fontsize=14)
I_ion = None
for i, name in enumerate(LIST_OF_CURRENTS):            
    I_x = model.current_response_info.get_current([name])
    if i==0 :
        I_ion = copy.copy(I_x)
        print(I_ion)
    else :
        I_ion += I_x
        
for i, name in enumerate(plot_li):    
    # ax.set_title('Simulation %d'%(simulationNo))
    axes[i].set_xlim(model.times.min(), model.times.max())
    # ax.set_ylim(ylim[0], ylim[1])
    axes[i].set_ylabel(name)  
    axes[i].set_xlabel('Time (ms)') 
    if name=='command_voltages':
        axes[i].plot( model.times, tr.command_voltages, label=name, c='b') 
    elif name=='I_ion':        
        axes[i].plot( model.times, I_ion, label=name, c='black')     
    else:        
        axes[i].plot( model.times, model.current_response_info.get_current([name]), label=name, c='black')     
    axes[i].legend()
    axes[i].grid()
plt.subplots_adjust(left=0.0, bottom=0.0, right=1.0, top=0.96, wspace=0.5, hspace=0.15)
plt.show()
# fig.savefig(os.path.join('Results', "ORD2011-JK-scipy"), dpi=100)

In [ ]:
protocol_li = []
for i in [0, 1, 2, 3, 0, 1, 3]:
    protocol_li.append(best_protocol.steps[i])
protocol = mod_protocols.VoltageClampProtocol(steps=protocol_li)

In [ ]:
model = None
if 'ORD2011' in trial_conditions:
    model = ORD2011(protocol)
elif 'BR1977' in trial_conditions:
    model = BR1977(protocol)  
    
tr = model_response.get_model_response_JK(model, protocol)

scale = 1
max_contributions = tr.current_response_info.\
            get_max_current_contributions(
                time=tr.t,
                window=window/scale,
                step_size=step_size/scale)
print(max_contributions)
tr.plot_currents_contribution(current, window=window, step_size=step_size, is_shown=True, title=current,
        saved_to=f'./ga_results/{trial_conditions}/{current}.svg')